# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-mini" # gpt-4o-mini or gpt-4o

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Create AI Foundry Project Client

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

# Extract Azure OpenAI client from the AI Foundry Project

In [3]:
from openai import AzureOpenAI # <<<<< NOT NEEDED FOR AZURE AI FOUNDRY SDK

# using explicit type hinting for IntelliSense
azure_openai_client: AzureOpenAI = project_client.inference.get_azure_openai_client(
    api_version = os.environ.get("AZURE_OPENAI_API_VERSION"),
)

# if NOT MI --> azure_openai_client.api_key = os.environ.get("AZURE_OPENAI_API_KEY") 

print(azure_openai_client.base_url)

https://mmai-hub04-ai-servicesfvye.openai.azure.com/openai/


# Create OpenAI Assistant Agent with CodeInterpreterTool as one of the tools

In [4]:
# First, create the code interpreter definitions...
code_interpreter_tool = {"type": "code_interpreter"}
code_interpreter_resources={"code_interpreter":{"file_ids":[]}}

# ...then, create the OpenAI Assistant Agent, attaching code interpreter
assistant_agent = azure_openai_client.beta.assistants.create(
  model=model_name,
  instructions="You are a helpful agent",
  tools=[code_interpreter_tool],
  tool_resources=code_interpreter_resources,
  temperature=1,
  top_p=1
)

# Create the thread and attach a new message to it

In [5]:
# Create a thread
thread = azure_openai_client.beta.threads.create()

print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = azure_openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    """,
)

azure_openai_client.beta.threads.messages.retrieve \
    (thread_id=thread.id, message_id=message.id)

Created thread: Thread(id='thread_7rKNxsfVOPwQ0inyuZdTAqj7', created_at=1737213175, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))



Message(id='msg_Y01WnCB72DyydQtNpL06R0Wu', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='\n    Could you please create a bar chart for the operating profit using \n    the following data and provide the file to me? \n    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, \n    Company D: $1.8 million\n    '), type='text')], created_at=1737213175, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_7rKNxsfVOPwQ0inyuZdTAqj7')

# Run the agent syncrhonously

In [6]:
%%time

# Run the agent
run = azure_openai_client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_agent.id
)
print(f"Run status: {run.status}")

# Looping until the run completes or fails
import time
while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = azure_openai_client.beta.threads.runs.retrieve\
        (thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")  

Run status: queued
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: in_progress
Run status: completed
CPU times: total: 62.5 ms
Wall time: 11.6 s


# Retrieve messages from the thread after the agent run execution

In [7]:
import openai

if run.status == 'completed':    
    messages = azure_openai_client.beta.threads.messages.list(thread_id=thread.id)
    print(f"Here are the {len(messages.data)} messages, starting with the most recent one:\n")
    i=0
    for m in messages.data:
        j =0
        i += 1
        print(f"\n===== MESSAGE {i} =====")
        for c in m.content:
            j += 1
            if (type(c) is openai.types.beta.threads.image_file_content_block.ImageFileContentBlock):
                print(f"\nCONTENT {j} (ImageFileContentBlock)--> image_file id: {c.image_file.file_id}")
            elif (type(c) is openai.types.beta.threads.text_content_block.TextContentBlock):
                print(f"\nCONTENT {j} (TextContentBlock) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")
else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

Here are the 2 messages, starting with the most recent one:


===== MESSAGE 1 =====

CONTENT 1 (ImageFileContentBlock)--> image_file id: assistant-XJt7kN0R0AGDWy2qcGuJXdYu

CONTENT 2 (TextContentBlock) --> Text: I have created the bar chart for the operating profit, and you can download it using the link below:

[Download Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)
>>> Annotation in MessageTextContent 2 of message 1: sandbox:/mnt/data/operating_profit_chart.png


===== MESSAGE 2 =====

CONTENT 1 (TextContentBlock) --> Text: 
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    


# Delete all assistants

In [8]:
assistants = azure_openai_client.beta.assistants.list().data
print (f"Nr. of existing agents that will be deleted: {len(assistants)}")

i = 0
for a in assistants:
    i += 1
    azure_openai_client.beta.assistants.delete(a.id)
    print(f"\n{i} - Deleted agent {a}")
    

Nr. of existing agents that will be deleted: 1

1 - Deleted agent Assistant(id='asst_USaATqCeq1nsXdzXdc7KHcTK', created_at=1737213175, description=None, instructions='You are a helpful agent', metadata={}, model='gpt-4o-mini', name=None, object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)


# HIC SUNT LEONES